In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
DOCS_TO_LOAD = "/content/drive/MyDrive/itmo_tasks/nlp/assets/data/annotated_tsv/train_test_lists.json" if IN_COLAB else "./assets/data/annotaed-tsv/train_test_lists.json"

DB_NAME_PREFIX = "toxicity"
DB_DIR = "/content/drive/MyDrive/itmo_tasks/nlp/assets/data/databases/vector_db_toxicity"


In [ ]:
import os
import re
import json
from pathlib import Path
import math

In [ ]:
os.makedirs(DB_DIR, exist_ok=True)

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from tqdm import tqdm

In [ ]:
class DocTokenizer():
    def __init__(self, lang: str = "russian") -> None:
        self.lang = lang

    def split_to_sentences(self, text: str) -> list[str]:

        sentences = re.split(r"(!+\?+)|(\?+!+)|(\.{2,})|(\?{2,})|(!{2,})|(\? )|(! )|(\. )", text)[::9]
        return sentences

    def split_to_words(self, sentence: str) -> list[str]:
        words = re.findall(r"\w+@\w+\.\w+|\+\d{1,3}-\d{3}-\d{3}-\d{2}-\d{2}|\w+", sentence)
        return words

    def doc_to_sents(self, text: str) -> list[list[str]]:

        sentences = self.split_to_sentences(text)
        result = []
        for s in sentences:
            sentence = []
            for w in self.split_to_words(s):
                w_processed = re.sub(r"[.!?,]$", "", w).lower()
                sentence.append(w_processed)

            result.append(sentence)

        return result

In [ ]:
class DocSplitter:
    def __init__(self, fragment_size: int = 100, overlap: int = 0, doc_tokenizer = None, token_splitter: str = " ") -> None:
        self.overlap = overlap
        self.doc_tokenizer = doc_tokenizer
        self.token_splitter = token_splitter
        self.fragment_size = fragment_size

    def _concat_sents(self, sents: list[list[str]]):
        return self.token_splitter.join([self.token_splitter.join(sent) for sent in sents])

    def split_doc(self, doc: list[list[str]] | str) -> list[str]:
        if isinstance(doc, str):
            doc = self.doc_tokenizer.doc_to_sents(doc)


        result = []
        sent_lens = [len(sent) for sent in doc]
        left_sent_id = 0
        while left_sent_id < len(sent_lens):
            right_sent_id = left_sent_id
            curr_frag_size = 0

            while curr_frag_size < self.fragment_size and right_sent_id < len(sent_lens):
                curr_frag_size += sent_lens[right_sent_id]
                right_sent_id += 1

            result.append(self._concat_sents(doc[left_sent_id:right_sent_id]))

            left_sent_id = right_sent_id

        return result



class Dataset:
    def __init__(self, paths_to_documents: list[str], doc_splitter: DocSplitter) -> None:
        self.paths_to_documents = paths_to_documents
        self.doc_splitter = doc_splitter
        self.docs = []
        self.metas = []

    def __len__(self):
        return len(self.docs)

    @staticmethod
    def _load_doc_tsv(path: str) -> list[list[str]]:
        sentences = []
        with open(path, mode="r") as f:
            f.readline()
            lines = "".join(f.readlines())
            sentences_raw = lines.split("\n\t\t\n")
            for sentence in sentences_raw:
                words = sentence.split("\n")
                if len(words) == 0 or words[0] == "":
                    continue
                tokens = list(map(lambda x: x.split("\t")[0], words))
                sentences.append(tokens)
        return sentences

    def _load_data_sync(self):
        metas = []
        for filepath in self.paths_to_documents:
            p = Path(filepath)
            meta = {
                "document": p.stem.strip(),
                "topic": p.parent.stem,
            }
            metas.append(meta)

        doc_sents = [self._load_doc_tsv(filepath) for filepath in tqdm(self.paths_to_documents, desc="Loading files")]

        for text, meta in tqdm(zip(doc_sents, metas), desc="Creating fragments"):
            frags = self.doc_splitter.split_doc(text)

            self.metas.extend([meta] * len(frags))
            self.docs.extend(frags)

    def _load_data(self, async_=False):
        if async_:
            # asyncio.run(self._load_data_coro())
            pass
        else:
            self._load_data_sync()

    def prefetch_dataset(self):
        if not self.docs:
            self._load_data()

    def reset_dataset(self):
        self.docs = []
        self.metas = []

    def update_dataset(self):
        self.reset_dataset()
        self.prefetch_dataset()

    def get_documents(self, batch_size: int = 1024):
        if not self.docs:
            self._load_data()

        l = 0
        while l < len(self.docs):
            r = min(l + batch_size, len(self.docs))
            yield self.docs[l:r], self.metas[l:r]
            l = r


In [ ]:

class EmbeddingFunction:
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"):
        self.model = SentenceTransformer(model_name)

    def __call__(self, input):
        return self.model.encode(input).tolist()


class VectorDB:
    def __init__(self, name_prefix, root_path, embeddnig_fn, distance_fn) -> None:
        self.client = chromadb.PersistentClient(path=root_path)
        self.distance_fn = distance_fn
        self.embedding_fn = embeddnig_fn
        self._collection_name = name_prefix + self.distance_fn

        self._get_or_create_collection()


    def _get_or_create_collection(self):
        self.database = self.client.get_or_create_collection(
            self._collection_name,
            metadata={"hnsw:space": self.distance_fn},
            embedding_function=self.embedding_fn
        )

    def _delete_collection(self):
        self.client.delete_collection(self._collection_name)

    def load_dataset(self, dataset: Dataset) -> None:
        batch_size = 128
        left_i = 0
        right_i = 0
        for texts, metas in tqdm(dataset.get_documents(batch_size=batch_size), total=math.ceil(len(dataset.docs) / batch_size), desc="loading dataset to the DB"):
            right_i = left_i + len(texts)
            self.database.add(
                documents=texts,
                metadatas=metas,
                ids=list(map(str, range(left_i, right_i))),
                # ids=list([f"{meta['topic']}/{meta['document']}" for meta in metas]),
            )
            left_i = right_i


    def query(self, query, n_results: int):
        return self.database.query(query_embeddings=self.embedding_fn(query), n_results=n_results)

    def clear(self):
        self._delete_collection()
        self._get_or_create_collection()



In [ ]:
with open(DOCS_TO_LOAD) as f:
    dataset_meta = json.load(f)

docs_paths = dataset_meta["test"]

In [ ]:
splitter = DocSplitter(doc_tokenizer=DocTokenizer())
dataset = Dataset(docs_paths, splitter)

In [ ]:
len(dataset.docs)

In [ ]:
database_cos = VectorDB("toxicity", DB_DIR, EmbeddingFunction(), "cosine")

In [ ]:
database_l2 = VectorDB("toxicity", DB_DIR, EmbeddingFunction(), "l2")

In [ ]:
database_ip = VectorDB("toxicity", DB_DIR, EmbeddingFunction(), "ip")

In [ ]:
dataset.update_dataset()

In [ ]:
database_cos.load_dataset(dataset)

In [ ]:
database_l2.load_dataset(dataset)

In [ ]:
database_ip.load_dataset(dataset)

In [ ]:
with open(DOCS_TO_LOAD) as f:
    dataset_meta = json.load(f)

# docs for the db
train_docs_paths = dataset_meta["train"]

dataset_train = Dataset(train_docs_paths, splitter)
dataset_train.update_dataset()
len(dataset_train)

In [ ]:
database_cos.load_dataset(dataset_train)
database_l2.load_dataset(dataset_train)
database_ip.load_dataset(dataset_train)

In [ ]:
phrases_to_test = (
    "Зачем ты что-то пишешь потому что могу у тебя бамбануло, хаха порвался с учётом ваших длинных комментариев- так и есть смеешься там как умственно отсталый Это как? Есть какие-либо критерии как смеются умственно отсталые? Или ты как обычно пишешь абстрактные вещи, смысл которые даже тебе не понятен?",
    "Нашёл как то работу отличную с зп в районе 50-60к, для меня это было просто супер, тк на старой я получал 15-30, обязанности все те же и условия лучше, собеседование и стажировку в 3 дня прошёл успешно я им понравился, но не пропустил сб, хотя до этого работы другой не было, а на этой косяков не было)",
    "В 2021 году астрономы добавили к списку потенциально обитаемых классов планет ещё один — так называемые гикеановские экзопланеты. Это название — производное от hydrogen (водород) и ocean (океан). По словам учёных, такие планеты — горячие, они полностью покрыты водой, а их атмосфера богата водородом. Одна из них — K2-18b, и о ней сегодня поговорим под катом.",
    "Similar to diffusion models, consistency models enable various data editing and manipulation applications in zero shot; they do not require explicit training to perform these tasks. For example, consistency models define a one-to-one mapping from a Gaussian noise vector to a data sample.",
    "Пилат поднял мученические глаза на арестанта и увидел, что солнце уже довольно высоко стоит над гипподромом, что луч пробрался в колоннаду и подползает к стоптанным сандалиям Иешуа, что тот сторонится от солнца.",
)

In [ ]:
from pprint import pprint

In [ ]:
def query_samples(samples, db: VectorDB, k=10):
    for sample in samples:
        results = db.query(sample, k)
        print("Sample is: ", sample[:20])
        print("Response is: ")
        pprint(results)

In [ ]:
print("TEST COSINE SIMILARITY")
query_samples(phrases_to_test, database_cos)

In [ ]:
print("TEST L2 SIMILARITY")
query_samples(phrases_to_test, database_l2)

In [ ]:
print("TEST IP SIMILARITY")
query_samples(phrases_to_test, database_ip)

In [ ]:
from ctransformers import AutoModelForCausalLM
import gradio as gr

In [ ]:
def get_rag_handler(model, vec_db):
    def rag_handler(msg, his):
        ctx = "\n".join(vec_db.query(msg, 1)["documents"][0])
        print(ctx)
        prompt = """Ответь на вопрос (Question), учитывая контекст (Context). Длина твоего ответа (Answer) не должна превышать 50 слов. Ответ должен быть на русском языке.

        Context: {ctx}

        Question: {qst}

        Answer: """.format(ctx=ctx, qst=msg)

        answ = model(prompt)
        return answ

    return rag_handler



In [ ]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

In [ ]:
demo = gr.ChatInterface(fn=get_rag_handler(model, database_cos), title="Чат")
demo.launch()